In [ ]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
gdf = gpd.read_file("laplata_cascourbano.kml", driver="KML")
print(gdf.head())


In [ ]:
# Chequear cuántas capas tiene el archivo kml
for layer in fiona.listlayers("laplata_cascourbano.kml"):
    print(layer)

In [ ]:
# Convertir el archivo kml a geojson
gdf.to_file("salida.geojson", driver='GeoJSON')

In [ ]:
# Plotear el archivo geojson
gdf = gpd.read_file("salida.geojson")
gdf.plot()

# Etapa 2 : Obtener datos de densidad poblacional de la ciudad de La Plata.

In [ ]:
# Dataset de densidad poblacional Argentina 2020 (1km resolución, ~4M puntos)
pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")

In [ ]:
# Función para extraer datos de una ciudad específica del dataset completo
def extract_city_data(city_name: str, bounds: dict) -> pd.DataFrame:

    df = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")
    
    city_data = df[
        (df['X'] >= bounds['x_min']) & 
        (df['X'] <= bounds['x_max']) & 
        (df['Y'] >= bounds['y_min']) & 
        (df['Y'] <= bounds['y_max'])
    ]
    
    # Guardar subset
    output_file = f"{city_name.lower().replace(' ', '_')}_population_2020.csv"
    city_data.to_csv(output_file, index=False)
    
    print("Filas extraídas:", f"{len(city_data):,}")
    
    return city_data

In [ ]:
# Coordenadas aproximadas para La Plata
la_plata_bounds = {
   'x_min': -58.2,
   'x_max': -57.7,
   'y_min': -35.2,
   'y_max': -34.7
}

# Extraer datos
la_plata_data = extract_city_data("La Plata", la_plata_bounds)


In [ ]:
# Leer el archivo CSV de densidad de población de La Plata
df = pd.read_csv('la_plata_population_2020.csv')
    
# Crear geometría de puntos usando X,Y como longitud,latitud
geometry = [Point(xy) for xy in zip(df['X'], df['Y'])]
    

In [ ]:
# Crear GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    
# Agregar columna de densidad con nombre descriptivo
gdf['densidad_poblacional_lp'] = gdf['Z']
gdf


In [ ]:
def export_to_geojson(gdf, output_path='la_plata_densidad.geojson'):
    
    gdf.to_file(output_path, driver='GeoJSON')
    print(f"Archivo exportado: {output_path}")
    return output_path

In [ ]:
export_to_geojson(gdf)

In [ ]:
# Visualización de la densidad poblacional de La Plata
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de dispersión geográfica
scatter = axes[0].scatter(
    gdf['X'], gdf['Y'], c=gdf['Z'],
    cmap='inferno', alpha=0.6, s=10
)
axes[0].set_xlabel('Longitud')
axes[0].set_ylabel('Latitud')
axes[0].set_title('Distribución Geográfica de Densidad Poblacional de La Plata')
plt.colorbar(scatter, ax=axes[0], label='Densidad')

# Histograma de densidades
axes[1].hist(gdf['Z'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Densidad Poblacional')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Densidad Poblacional de La Plata')

plt.tight_layout()
plt.savefig('analisis_densidad_la_plata.png', dpi=300, bbox_inches='tight')
plt.show()


# Etapa 3 : Obtener datos catastrales/domiciliares de la ciudad de La Plata.